# Two-dimensional Diffusion Equation Solver Using Finite Volume Method
### Laura Shi

In [54]:
import scipy as sc
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

# Set some parameters for plotting
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 16
%matplotlib inline

## Input deck values

In [55]:
n = 10 #size for n x n grid
m = 10

D = np.ones((n, m))*10 #diffusion coeffient grid
splitval = int((1/2)*n) #split value for the hald
D[:, :splitval] = 20 #left regime

Sigma_a = np.ones((n, m))*100
Sigma_a[:, :splitval] = 200 #left regime different material properties

Source = np.ones((n, m))*40 #source term
Source[:, :splitval] = 80 #source term
delta = np.ones((n, )) #x spacing
eps = np.ones((n, ))*2 #y spacing

display(Sigma_a, D, Source, delta, eps)

array([[200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.],
       [200., 200., 200., 200., 200., 100., 100., 100., 100., 100.]])

array([[20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 10., 10., 10., 10., 10.]])

array([[80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.],
       [80., 80., 80., 80., 80., 40., 40., 40., 40., 40.]])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [56]:
def calcV(i, j):
    return (1/4)* delta[i]*eps[j]

In [57]:
def calcSigma_a(i, j):
    av = 0
    i_limit = min(Sigma_a.shape[0], i+2)
    j_limit = min(Sigma_a.shape[1], j+2)
    for x in range(i, i_limit):
        for y in range(j, j_limit):
            av = av + Sigma_a[x][y]*calcV(x, y)
    return av

## Creating the $A$ Matrix

In [58]:
#UPDATE VALUES, right now just uses dummy values to make sure displaying correctly
def aL(i, j):
    return -(D[i][j]*eps[j] + D[i][j+1]*eps[j+1])/(2*delta[i])
def aR(i, j):
    return -(D[i+1][j]*eps[j] + D[i+1][j+1]*eps[j+1])/(2*delta[i+1])
def aB(i, j):
    return -(D[i][j]*delta[i] + D[i+1][j+1]*delta[i+1])/(2*eps[j])
def aT(i, j):
    return -(D[i][j+1]*delta[i] + D[i+1][j+1]*delta[i+1])/(2*eps[j+1])
def aC(i, j):
    return calcSigma_a(i, j)- (aL(i, j) + aR(i, j) + aB(i, j) + aT(i, j)) 

In [59]:
#submatrix creation
def buildCenter(n, m): #right now produces a square n x n submatrix
    #n x n is dimension of individual matrix, m is the relative matrix number
    A = np.zeros((n, n)) 
    A[0, 0], A[0, 1] = aC(0, m), aR(0, m) #first line
    for j in range(1, (n-1)):
        A[j, (j-1)], A[j, j], A[j, (j+1)] = aL(j, m), aC(j, m), aR(j, m) #middle lines
    A[(n-1), (n-2)], A[(n-1), (n-1)] = aL(n-1, m), aC((n-1), m) #last line
    return A

def buildBot(n, m):
    A = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            if (i == j):
                A[i][j] = aB(j, m)
    return A

def buildTop(n, m):
    A = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            if (i == j):
                A[i][j] = aT(j, m)
    return A

In [60]:
def buildFullA(n): #assuming n is greater than or equal to 2
    A = np.zeros((n**2, n**2))
    A[0:n, 0:n], A[0:n, n:2*n] = buildCenter(n, 0), buildTop(n, 0)
    for k in range(1, (n-1)): 
        A[(k*n):(k+1)*n, (k-1)*n:(k*n)] = buildBot(n, k) #dimensions n, k is value to invoke
        A[(k*n):(k+1)*n, (k*n):(k+1)*n] = buildCenter(n, k)
        A[(k*n):(k+1)*n, (k+1)*n:(k+2)*n] = buildTop(n, k)
    A[(n-1)*n:n*n, (n-2)*n:(n-1)*n]= buildBot(n, (n-1))
    A[(n-1)*n:n*n, (n-1)*n:n*n] = buildCenter(n, (n-1))
    return A

In [61]:
A = buildFullA(9)
display(A)

array([[500., -40.,   0., ...,   0.,   0.,   0.],
       [-40., 500., -40., ...,   0.,   0.,   0.],
       [  0., -40., 500., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 250., -20.,   0.],
       [  0.,   0.,   0., ..., -20., 250., -20.],
       [  0.,   0.,   0., ...,   0., -20., 250.]])

## Creating the Source Term $\vec{S}$

\begin{equation}
\underbrace{\begin{pmatrix}
\begin{pmatrix}
a_{C}^{00} & a_{R}^{00} & 0 \\
a_{L}^{10} & a_{C}^{10} & a_{R}^{10} \\
0            & a_{L}^{20} & a_{C}^{20}
\end{pmatrix} 
&
\begin{pmatrix}
a_{T}^{00} & 0 & 0 \\
0 & a_{T}^{10} & 0 \\
0 & 0 & a_{T}^{20}
\end{pmatrix}
&
\begin{pmatrix}
 & & \\
 & \vec{0} & \\
 & & 
\end{pmatrix} \\
%--------------------
\begin{pmatrix}
a_{B}^{01} & 0 & 0 \\
0 & a_{B}^{11} & 0 \\
0 & 0 & a_{B}^{21}
\end{pmatrix}
&
\begin{pmatrix}
a_{C}^{01} & a_{R}^{01} & 0 \\
a_{L}^{11} & a_{C}^{11} & a_{R}^{11} \\
0            & a_{L}^{21} & a_{C}^{21}
\end{pmatrix}
&
\begin{pmatrix}
a_{T}^{01} & 0 & 0 \\
0 & a_{T}^{11} & 0 \\
0 & 0 & a_{T}^{21}
\end{pmatrix} \\
%--------------------
\begin{pmatrix}
 & & \\
 & \vec{0} & \\
 & & 
\end{pmatrix} &
\begin{pmatrix}
a_{B}^{02} & 0 & 0 \\
0 & a_{B}^{12} & 0 \\
0 & 0 & a_{B}^{22}
\end{pmatrix}
&
\begin{pmatrix}
a_{C}^{02} & a_{R}^{02} & 0 \\
a_{L}^{12} & a_{C}^{12} & a_{R}^{12} \\
0            & a_{L}^{22} & a_{C}^{22}
\end{pmatrix} \\
\end{pmatrix}}_{\vec{A}}
%--------------------
%
\underbrace{\begin{pmatrix} \phi_{0,0} \\ \phi_{1,0} \\ \phi_{2,0} \\ \\ \phi_{0,1} \\ \phi_{1,1} \\ \phi_{2,1} \\ \\ \phi_{0,2}\\ \phi_{1,2} \\  \phi_{2,2} \end{pmatrix}}_{\vec{\phi}} =
%
\underbrace{\begin{pmatrix} S_{00} \\ S_{10} \\ S_{20} \\ \\ S_{01} \\ S_{11} \\ S_{21} \\ \\ S_{02} \\ S_{12} \\  S_{22} \end{pmatrix}}_{\vec{S}} \nonumber
\end{equation}

Where $$S_{ij} = { S_{i,j} V_{i,j} + S_{i+1,j} V_{i+1,j} + S_{i+1,j+1} V_{i+1,j+1} + S_{i,j+1} V_{i,j+1} } $$
and
$$V_{i,j} = \frac{1}{4}\delta_i \epsilon_j \:, \quad V_{i+1,j} = \frac{1}{4}\delta_{i+1} \epsilon_{j} \:, \quad V_{i+1,j+1} = \frac{1}{4}\delta_{i+1} \epsilon_{j+1} \:, \quad V_{i,j+1} = \frac{1}{4}\delta_{i} \epsilon_{j+1} \:.$$

In [62]:
#NEED TO FIX WHEN FIGURE OUT ij business
def calcSource(i, j):
    av = 0
    i_limit = min(Source.shape[0], i+2)
    j_limit = min(Source.shape[1], j+2)
    for x in range(i, i_limit):
        for y in range(j, j_limit):
            av = av + Source[x][y]*calcV(x, y) #issue is overflow with the calc source method at end points
    return av

In [63]:
def createSvec(n, m):
    Svec = np.zeros(((n*m), 1)) #initialize array vector
    for j in range(0, m):
        temp = np.array([calcSource(i, j) for i in range(0, n)]).reshape(n, 1)
        Svec[j*n: (j+1)*(n)] = temp
    return Svec

In [64]:
netSource = createSvec(9, 9)
netSource.shape

(81, 1)

## Perform Iterative Methods to Solve for the Flux, $\vec{\phi}$

In [65]:
#general convergence checker that is general per method
def performIteration(A, b, xk, tol, method):
    iter_count = 0;
    err = np.inf
    while err > tol:
        X_new = method(A, b, xk)
        err = np.linalg.norm((X_new - xk)) #error is difference between previous iteration!
        iter_count = iter_count + 1
        xk = X_new
    return xk, err, iter_count

In [66]:
#perform one Jacobian iteration given A, b, and xk
#used as inner function for convergence checker
def jacobi(A, b, xk):
    newX = []
    n = len(b)
    for i in range(0, n):
        summation = 0
        for j in range(0, n):
            if (j !=i):
                summation = summation + A[i][j]*xk[j]
        xi = (1/(A[i][i]))* (b[i] - summation)
        newX.append(xi)
    return np.array(newX)

In [79]:
n = 9
A = buildFullA(n)
b = createSvec(n, n)
x0 = np.zeros(n*n) 
tol = 10**(-6)

xk, err, count = performIteration(A, b, x0, tol, jacobi)
#print('solution vector Jacobi:\n', xk)
print('iteration count:', count)
print('error:', err)

iteration count: 10
error: 8.11981301678759e-07


In [69]:
def gs(A, b, xk):
    newX = []
    n = len(b)
    for i in range(0, n):
        sum1 = 0
        sum2 = 0
        for j in range(0, n):
            if (j < i):
                sum1= sum1 + A[i][j]*newX[j]
            elif (j > i):
                sum2 = sum2 + A[i][j]*xk[j]
        
        xi = (1/(A[i][i]))* (b[i] - sum1 - sum2)
        newX.append(xi)
    return np.array(newX)

In [71]:
xk, err, count = performIteration(A, b, x0, tol, gs)
#print('solution vector GS:', xk)
print('iteration count:', count)
print('error:', err)

iteration count: 8
error: 3.1709448301467257e-07


In [72]:
def performIterationSOR(A, b, xk, w, tol):
    iter_count = 0;
    err = np.inf
    while err > tol:
        X_new = sor(A, b, xk, w)
        err = np.linalg.norm((X_new - xk)) #error is difference between previous iteration!
        iter_count = iter_count + 1
        xk = X_new
    return xk, err, iter_count

In [73]:
def sor(A, b, xk, w):
    newX = []
    n = len(b)
    for i in range(0, n):
        sum1 = 0
        sum2 = 0
        for j in range(0, n):
            if (j < i):
                sum1= sum1 + A[i][j]*newX[j]
            elif (j > i):
                sum2 = sum2 + A[i][j]*xk[j]
        xi = (1- w)*xk[i] + (w/(A[i][i]))* (b[i] - sum1 - sum2)
        newX.append(xi)
    return np.array(newX)

In [76]:
w = 1.15
xk, err, count = performIterationSOR(A, b, x0, w, tol)
#print('solution vector SOR w = 1.15:\n', xk)
print('iteration count:', count)
print('error:', err)

solution vector SOR w = 1.15:
 [[0.35835126]
 [0.38782762]
 [0.39025318]
 [0.39045275]
 [0.39046783]
 [0.39045275]
 [0.39025318]
 [0.38782762]
 [0.35835126]
 [0.36625265]
 [0.39696333]
 [0.39953754]
 [0.39975313]
 [0.39976967]
 [0.39975313]
 [0.39953754]
 [0.39696332]
 [0.36625265]
 [0.36642791]
 [0.39717781]
 [0.39975821]
 [0.39997463]
 [0.39999125]
 [0.39997463]
 [0.39975821]
 [0.39717781]
 [0.36642791]
 [0.36643182]
 [0.39718284]
 [0.39976345]
 [0.3999799 ]
 [0.39999653]
 [0.3999799 ]
 [0.39976345]
 [0.39718284]
 [0.36643182]
 [0.36643191]
 [0.39718295]
 [0.39976357]
 [0.39998002]
 [0.39999665]
 [0.39998003]
 [0.39976357]
 [0.39718295]
 [0.36643191]
 [0.36643182]
 [0.39718284]
 [0.39976345]
 [0.3999799 ]
 [0.39999653]
 [0.3999799 ]
 [0.39976345]
 [0.39718284]
 [0.36643182]
 [0.36642791]
 [0.39717781]
 [0.39975821]
 [0.39997463]
 [0.39999125]
 [0.39997463]
 [0.39975821]
 [0.39717781]
 [0.36642791]
 [0.36625265]
 [0.39696333]
 [0.39953754]
 [0.39975313]
 [0.39976968]
 [0.39975312]
 [0